In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import analysis_util
import plotly.express as px
sns.set_theme()

In [29]:
subreddit = 'wallstreetbets'
period = '2015-01-01-2022-01-01'

path_post='../data/raw/'+ subreddit +'_'+  period +'/'+ subreddit + "_posts.csv"
path_comment = '../data/raw/'+ subreddit +'_'+  period +'/'+ subreddit + "_comments.csv"

df_comments=analysis_util.read_comments(path_comment)
df_post=analysis_util.read_posts(path_post)
# df_all_nodes=analysis_util.get_all_nodes(df_post,df_comments)
    
#     G_di = analysis_util.get_DiGraph(df_post,df_comments,df_all_nodes)
#     G=analysis_util.get_graph(df_post,df_comments,df_all_nodes)
#     G_big=analysis_util.get_biggest_component(G)
    
# df_comment_post = analysis_util.get_comment_post_date(df_comments,df_post)
# df_authors = analysis_util.get_authors(G,df_all_nodes,df_comments,df_post,df_comment_post)

#     df_comment_dict[subreddit] = df_comments
#     df_post_dict[subreddit] = df_post
#     df_all_nodes_dict[subreddit] = df_all_nodes
#     G_dict[subreddit] = G_big
#     df_authors_dict[subreddit] = df_authors
#     G_di_dict[subreddit] = G_di

In [30]:
df_comment_post
#
# sns.scatterplot(data=df_comment_post, x="post_date", y="comment_date", hue="post_id", alpha=0.5)

px.histogram(df_comment_post, x="date",nbins=800)
# sns.histplot(data=df_comment_post, x="date")
#calendar heatmap